In [1]:
import sys
sys.path.append('../..')

In [2]:
from src.recommenders.baseline_recommender import BaselineRecommender
from src.evaluation import average_precision

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
books_df = pd.read_csv('../../data/Books.csv', delimiter=';')
ratings_df = pd.read_csv('../../data/Ratings.csv', delimiter=';')
users_df = pd.read_csv('../../data/Users.csv', delimiter=';')

/var/folders/s0/4m_t_82936ggv17n0g0t7_dr0000gn/T/ipykernel_60324/2786065433.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('../../data/Users.csv', delimiter=';')


In [5]:
train_ratings, test_ratings = train_test_split(ratings_df, test_size=0.3, random_state=42)

In [6]:
model = BaselineRecommender()
model.fit(items=books_df, users=users_df, ratings=train_ratings)

In [7]:
test_users = test_ratings['User-ID'].unique()

In [8]:
best_items = model.predict(test_users)

In [9]:
len(test_users)

53334

In [10]:
from tqdm import tqdm

In [11]:

rating_mask = test_ratings['Rating'] >= 5

mean_ap = 0
for i in tqdm(range(len(test_users))):
    user_mask = test_ratings['User-ID'] == test_users[i]
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    if len(relevant_items) == 0:
        continue
    ap = average_precision(best_items[i], relevant_items['ISBN'].to_numpy())
    mean_ap += ap

mean_ap /= len(test_users)
mean_ap

100%|██████████| 53334/53334 [00:23<00:00, 2291.30it/s]


0.000632279760813595